In [1]:
import ccxt
from datetime import time
import asyncio

### API Keys Setup
#### Required Credentials
- The API credentials usually include the following:

    - **apiKey**. This is your public API Key and/or Token. This part is non-secret, it is included in your request header or body and sent over HTTPS in open text to identify your request. It is often a string in Hex or Base64 encoding or an UUID identifier.
    - **secret**. This is your private key. Keep it secret, don't tell it to anybody. It is used to sign your requests locally before sending them to exchanges. The secret key does not get sent over the internet in the request-response process and should not be published or emailed. It is used together with the nonce to generate a cryptographically strong signature. That signature is sent with your public key to authenticate your identity. Each request has a unique nonce and therefore a unique cryptographic signature.
    - **uid**. Some exchanges (not all of them) also generate a user id or uid for short. It can be a string or numeric literal. You should set it, if that is explicitly required by your exchange. See their docs for details.
    - **password**. Some exchanges (not all of them) also require your password/phrase for trading. You should set this string, if that is explicitly required by your exchange. See their docs for details.

#### Credential Validation
- For checking if the user has supplied all the required credentials the Exchange base class has a method called **exchange.checkRequiredCredentials()** or **exchange.check_required_credentials()**. Calling that method will throw an **AuthenticationError**, if some of the credentials are missing or empty. The **Exchange** base class also has property **exchange.requiredCredentials** that allows a user to see which credentials are required for this or that exchange, as shown below:

In [ ]:
exchange = ccxt.binance()
print(exchange.requiredCredentials())  # print required credentials

In [ ]:
exchange.check_required_credentials()   # raises AuthenticationError

#### Configuring API Keys
- To set up an exchange for trading just assign the API credentials to an existing exchange instance or pass them to exchange contructor upon instantiation, like so:

In [ ]:
# any time
bitfinex = ccxt.bitfinex()
bitfinex.apiKey = 'YOUR_BFX_API_KEY'
bitfinex.secret = 'YOUR_BFX_SECRET_KEY'

In [ ]:
# Upon instantiation
hitbtc = ccxt.hitbtc({
    'apiKey': 'YOUR_BFX_API_KEY',
    'secret': 'YOUR_BFX_SECRET_KEY'
})

In [ ]:
# from variable id
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET'
})

#### Overriding Nonce

In [ ]:
# 1: the shortest
binance = ccxt.binance({'nonce': ccxt.Exchange.milliseconds})

# 2: custom nonce
class MyKraken(ccxt.kraken):
    n = 1
    def nonce(self):
        return self.n += 1

# 3: milliseconds nonce
class MyBitfinex(ccxt.bitfinex):
    def nonce(self):
        return self.milliseconds()

# 4: milliseconds nonce inline
hitbtc = ccxt.hitbtc({'nonce': lambda: int(time.time()) * 1000})

# 5: milliseconds nonce
acx = ccxt.acx({'nonce': lambda: ccxt.Exchange.milliseconds()})

### Accounts
- You can get all the accounts associated with a profile by using the **fetchAccounts()** method

#### Accounts Structure
- The **fetchAccounts()** method will return a structure like shown below:

In [ ]:
accounts_structure = [
    {
        id: "s32kj302lasli3930",
        type: "main",
        name: "main",
        code: "USDT",
        info: { ... }
    },
    {
        id: "20f0sdlri34lf90",
        name: "customAccount",
        type: "margin",
        code: "USDT",
        info: { ... }
    },
    {
        id: "4oidfk40dadeg4328",
        type: "spot",
        name: "spotAccount32",
        code: "BTC",
        info: { ... }
    },
    ...
]

### Account Balance
- To query for balance and get the amount of funds available for trading or funds locked in orders, use the **fetchBalance** method:
#### Parameters
- **params** (Dictionary) Extra parameters specific to the exchange API endpoint (e.g. {"currency": "usdt"})
#### Returns
A **balance structure**

In [ ]:
balance_structure = {
    'info':  { ... },    # the original untouched non-parsed reply with details
    'timestamp': 1499280391811, # Unix Timestamp in milliseconds (seconds * 1000)
    'datetime': '2017-07-05T18:47:14.692Z', # ISO8601 datetime string with milliseconds

    #-------------------------------------------------------------------------
    # indexed by availability of funds first, then by currency

    'free':  {           # money, available for trading, by currency
        'BTC': 321.00,   # floats...
        'USD': 123.00,
        
    },

    'used':  { ... },    # money on hold, locked, frozen, or pending, by currency

    'total': { ... },    # total (free + used), by currency

    #-------------------------------------------------------------------------
    # indexed by currency first, then by availability of funds

    'BTC':   {           # string, three-letter currency code, uppercase
        'free': 321.00,   # float, money available for trading
        'used': 234.00,  # float, money on hold, locked, frozen or pending
        'total': 555.00, # float, total balance (free + used)
    },

    'USD':   {           # ...
        'free': 123.00,   # ...
        'used': 456.00,
        'total': 579.00,
    },

    
}

In [ ]:
print(exchange.fetch_balance())

### Querrying Orders
- The list of methods for querying orders consists of the following:

    - fetchCanceledOrders (symbol = undefined, since = undefined, limit = undefined, params = {})
    - fetchClosedOrder (id, symbol = undefined, params = {})
    - fetchClosedOrders (symbol = undefined, since = undefined, limit = undefined, params = {})
    - fetchOpenOrder (id, symbol = undefined, params = {})
    - fetchOpenOrders (symbol = undefined, since = undefined, limit = undefined, params = {})
    - fetchOrder (id, symbol = undefined, params = {})
    - fetchOrders (symbol = undefined, since = undefined, limit = undefined, params = {})

In [ ]:
# look into the .has property of the exchange to check above available querying methods
id = 'binance'
ex_class = getattr(ccxt, id)()
print(ex_class.has)

**NOTE**: not all exchanges provide means for filtering the lists of trades and orders by starting time, so, the support for since  and limit is exchange-specific. However, most exchanges do provide at least some alternative for "pagination" and "scrolling" which can be overrided with extra params argument.

#### By Order Id
- To get the details of a particular order by its id, use the fetchOrder() / fetch_order() method. Some exchanges also require a symbol even when fetching a particular order by id.

- The signature of the fetchOrder/fetch_order method is as follows:

In [ ]:
if exchange.has['fetchOrder']:
    cursor = 0
    while True:
        id = 'binance'
        symbol = 'BTC/USDT'
        params = {
            'cursor': cursor
        }
        order = await exchange.fetch_order(id, symbol=symbol, params=params)
        print(order)

#### Single Order
- Below are examples of using the fetchOrder method to get order info from an authenticated exchange instance:

In [ ]:
if exchange.has['fetchOrder']:
    order = exchange.fetch_order(id)
    print(order)

In [ ]:
# synchronous
import asyncio
import ccxt.async_support as ccxt

if exchange.has['fetchOrder']:
    order = asyncio.run(exchange.fetch_order(id))
    print(order)

#### All Orders

In [ ]:
if exchange.has['fetchOrders']:
    cursor = 0
    all_orders = []
    while True:
        since = exchange.milliseconds() * 60 * 60 * 1000   # last 1 hour
        limit = 10
        params = {
            'cursor': cursor
        }
        orders = exchange.fetchOrders(symbol=symbol, since=since, limit=limit, params=params)
        if len(orders) == 0:
            break
        else:
            cursor = exchange.last_response_headers['CB-AFTER']
            all_orders += orders